In [6]:
import heapq
from typing import List, Tuple, Set, Dict
import time

class Node:
    def __init__(self, state: Tuple[int, ...], parent=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.g = g
        self.h = h
        self.f = g + h
    
    def __lt__(self, other):
        return self.f < other.f or (self.f == other.f and self.h < other.h)

class PuzzleSolver:
    def __init__(self, initial_state: Tuple[int, ...], goal_state: Tuple[int, ...], heuristic='manhattan'):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.heuristic_type = heuristic
        self.expanded = 0
        self.generated = 0
        self.size = int(len(initial_state) ** 0.5)
        self.goal_positions = {val: i for i, val in enumerate(goal_state)}
    
    def manhattan_distance(self, state: Tuple[int, ...]) -> int:
        distance = 0
        for i, val in enumerate(state):
            if val != 0:
                goal_pos = self.goal_positions[val]
                current_row, current_col = i // self.size, i % self.size
                goal_row, goal_col = goal_pos // self.size, goal_pos % self.size
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance
    
    def misplaced_tiles(self, state: Tuple[int, ...]) -> int:
        return sum(1 for i, val in enumerate(state) if val != self.goal_state[i])
    
    def get_heuristic(self, state: Tuple[int, ...]) -> int:
        if self.heuristic_type == 'manhattan':
            return self.manhattan_distance(state)
        else:
            return self.misplaced_tiles(state)
    
    def get_neighbors(self, state: Tuple[int, ...]) -> List[Tuple[int, ...]]:
        neighbors = []
        zero_pos = state.index(0)
        zero_row, zero_col = zero_pos // self.size, zero_pos % self.size
        
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_row, new_col = zero_row + dr, zero_col + dc
            if 0 <= new_row < self.size and 0 <= new_col < self.size:
                new_pos = new_row * self.size + new_col
                state_list = list(state)
                state_list[zero_pos], state_list[new_pos] = state_list[new_pos], state_list[zero_pos]
                neighbors.append(tuple(state_list))
                self.generated += 1
        
        return neighbors
    
    def reconstruct_path(self, node: Node) -> List[Tuple[int, ...]]:
        path = []
        current = node
        while current:
            path.append(current.state)
            current = current.parent
        return path[::-1]
    
    def solve(self) -> Tuple[List[Tuple[int, ...]], Dict]:
        if self.initial_state == self.goal_state:
            return [self.initial_state], {'expanded': 0, 'generated': 0}
        
        open_heap = []
        open_dict = {}
        closed_set: Set[Tuple[int, ...]] = set()
        
        initial_h = self.get_heuristic(self.initial_state)
        initial_node = Node(self.initial_state, None, 0, initial_h)
        heapq.heappush(open_heap, (initial_node.f, initial_node.h, id(initial_node), initial_node))
        open_dict[self.initial_state] = initial_node
        self.generated += 1
        
        while open_heap:
            while open_heap and open_heap[0][3].state in closed_set:
                heapq.heappop(open_heap)
            
            if not open_heap:
                break
            
            _, _, _, current = heapq.heappop(open_heap)
            
            if current.state in closed_set:
                continue
            
            if current.state == self.goal_state:
                path = self.reconstruct_path(current)
                return path, {'expanded': self.expanded, 'generated': self.generated}
            
            closed_set.add(current.state)
            del open_dict[current.state]
            self.expanded += 1
            
            for neighbor_state in self.get_neighbors(current.state):
                if neighbor_state not in closed_set:
                    h = self.get_heuristic(neighbor_state)
                    neighbor_node = Node(neighbor_state, current, current.g + 1, h)
                    if neighbor_state not in open_dict or neighbor_node.f < open_dict[neighbor_state].f:
                        open_dict[neighbor_state] = neighbor_node
                        heapq.heappush(open_heap, (neighbor_node.f, neighbor_node.h, id(neighbor_node), neighbor_node))
        
        return [], {'expanded': self.expanded, 'generated': self.generated}

In [7]:
initial_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15)
goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

solver = PuzzleSolver(initial_state, goal_state, heuristic='manhattan')
path, stats = solver.solve()

print(f"Path length: {len(path)}")
print(f"Expanded nodes: {stats['expanded']}")
print(f"Generated nodes: {stats['generated']}")

Path length: 2
Expanded nodes: 1
Generated nodes: 4


In [8]:
def visualize_state(state, size=4):
    for i in range(size):
        print(' '.join(f'{state[i*size+j]:2d}' if state[i*size+j] != 0 else ' .' for j in range(size)))
    print()

print("Initial state:")
visualize_state(path[0])
print("Final state:")
visualize_state(path[-1])
print(f"Solution steps: {len(path) - 1}")

Initial state:
 1  2  3  4
 5  6  7  8
 9 10 11 12
13 14  . 15

Final state:
 1  2  3  4
 5  6  7  8
 9 10 11 12
13 14 15  .

Solution steps: 1


In [9]:
heuristics = ['manhattan', 'misplaced']
results = {}

for h_type in heuristics:
    solver = PuzzleSolver(initial_state, goal_state, heuristic=h_type)
    start_time = time.time()
    path_h, stats_h = solver.solve()
    elapsed = time.time() - start_time
    results[h_type] = {
        'path_length': len(path_h),
        'expanded': stats_h['expanded'],
        'generated': stats_h['generated'],
        'time': elapsed
    }

print("Performance Comparison:")
print(f"{'Heuristic':<15} {'Path':<8} {'Expanded':<12} {'Generated':<12} {'Time(s)':<10}")
for h_type, result in results.items():
    print(f"{h_type:<15} {result['path_length']:<8} {result['expanded']:<12} {result['generated']:<12} {result['time']:<10.4f}")

Performance Comparison:
Heuristic       Path     Expanded     Generated    Time(s)   
manhattan       2        1            4            0.0000    
misplaced       2        1            4            0.0000    


In [10]:
test_cases = [
    ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15), "Case 1: 0 at position 14"),
    ((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 13, 14, 15), "Case 2: 0 at position 12"),
]

goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

print("Multiple Test Cases:")
print(f"{'Case':<25} {'Path':<8} {'Expanded':<12} {'Generated':<12} {'Time(s)':<10}")
for initial_state, case_name in test_cases:
    solver = PuzzleSolver(initial_state, goal_state, heuristic='manhattan')
    start_time = time.time()
    path_t, stats_t = solver.solve()
    elapsed = time.time() - start_time
    print(f"{case_name:<25} {len(path_t):<8} {stats_t['expanded']:<12} {stats_t['generated']:<12} {elapsed:<10.4f}")

Multiple Test Cases:
Case                      Path     Expanded     Generated    Time(s)   
Case 1: 0 at position 14  2        1            4            0.0000    
Case 2: 0 at position 12  4        3            9            0.0000    
